# Projeto: Classificação de Dados End-to-End

Neste projeto prático, você irá desenvolver um pipeline completo de Machine Learning, desde a coleta de dados até a avaliação do modelo.

## Objetivo do Projeto

Criar um modelo de classificação para prever se um cliente irá comprar um produto com base em suas características demográficas e comportamentais.

## Etapas do Projeto

1. Importação e exploração de dados
2. Análise exploratória de dados (EDA)
3. Pré-processamento e feature engineering
4. Divisão dos dados
5. Treinamento de múltiplos modelos
6. Avaliação e comparação de modelos
7. Otimização de hiperparâmetros
8. Validação final
9. Interpretação dos resultados

In [ ]:
# Importações necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

import warnings
warnings.filterwarnings('ignore')

## 1. Geração e Exploração de Dados

In [ ]:
# Gerando um dataset sintético para o projeto
np.random.seed(42)
n_samples = 2000

dados = pd.DataFrame({
    'idade': np.random.randint(18, 70, n_samples),
    'salario': np.random.randint(20000, 150000, n_samples),
    'tempo_site': np.random.randint(1, 100, n_samples),
    'paginas_visitadas': np.random.randint(1, 50, n_samples),
    'genero': np.random.choice(['M', 'F'], n_samples),
    'pais': np.random.choice(['Brasil', 'EUA', 'Canada', 'Reino Unido'], n_samples),
})

# Criando a variável alvo com alguma lógica
dados['comprou'] = ((dados['tempo_site'] > 30) & (dados['salario'] > 60000)).astype(int)

# Adicionando algum ruído
noise_indices = np.random.choice(dados.index, size=int(0.1 * len(dados)), replace=False)
dados.loc[noise_indices, 'comprou'] = 1 - dados.loc[noise_indices, 'comprou']

print("Dataset criado com sucesso!")
print(f"\nShape: {dados.shape}")
print("\nPrimeiras linhas:")
dados.head()

## 2. Análise Exploratória de Dados (EDA)

In [ ]:
# Informações básicas
print("Informações do Dataset:")
print(dados.info())

print("\nEstatísticas Descritivas:")
print(dados.describe())

print("\nValores Ausentes:")
print(dados.isnull().sum())

print("\nDistribuição da Variável Alvo:")
print(dados['comprou'].value_counts())
print(f"\nPercentual de compradores: {dados['comprou'].mean():.2%}")

In [ ]:
# Visualizações
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Distribuição de idade
dados['idade'].hist(ax=axes[0, 0], bins=20, edgecolor='black')
axes[0, 0].set_title('Distribuição de Idade')

# Distribuição de salário
dados['salario'].hist(ax=axes[0, 1], bins=20, edgecolor='black')
axes[0, 1].set_title('Distribuição de Salário')

# Tempo no site
dados['tempo_site'].hist(ax=axes[0, 2], bins=20, edgecolor='black')
axes[0, 2].set_title('Tempo no Site')

# Distribuição por gênero
dados['genero'].value_counts().plot(kind='bar', ax=axes[1, 0], edgecolor='black')
axes[1, 0].set_title('Distribuição por Gênero')

# Distribuição por país
dados['pais'].value_counts().plot(kind='bar', ax=axes[1, 1], edgecolor='black')
axes[1, 1].set_title('Distribuição por País')
axes[1, 1].tick_params(axis='x', rotation=45)

# Distribuição da variável alvo
dados['comprou'].value_counts().plot(kind='bar', ax=axes[1, 2], edgecolor='black')
axes[1, 2].set_title('Distribuição: Comprou?')
axes[1, 2].set_xticklabels(['Não', 'Sim'], rotation=0)

plt.tight_layout()
plt.show()

## 3. Pré-processamento

In [ ]:
# Codificando variáveis categóricas
dados_processados = dados.copy()

# One-hot encoding para 'genero' e 'pais'
dados_processados = pd.get_dummies(dados_processados, columns=['genero', 'pais'], drop_first=True)

print("Dados após pré-processamento:")
print(dados_processados.head())
print(f"\nNovas features: {dados_processados.columns.tolist()}")

## 4. Divisão dos Dados

In [ ]:
# Separando features e target
X = dados_processados.drop('comprou', axis=1)
y = dados_processados['comprou']

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")
print(f"\nDistribuição no treino: {y_train.value_counts().to_dict()}")
print(f"Distribuição no teste: {y_test.value_counts().to_dict()}")

## 5. Treinamento de Múltiplos Modelos

In [ ]:
# Definindo os modelos
modelos = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True)
}

# Treinando e avaliando cada modelo
resultados = {}

for nome, modelo in modelos.items():
    print(f"\nTreinando {nome}...")
    modelo.fit(X_train_scaled, y_train)
    y_pred = modelo.predict(X_test_scaled)
    y_pred_proba = modelo.predict_proba(X_test_scaled)[:, 1]
    
    # Calculando métricas
    resultados[nome] = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_pred_proba)
    }
    
    print(f"Acurácia: {resultados[nome]['accuracy']:.4f}")
    print(f"Precision: {resultados[nome]['precision']:.4f}")
    print(f"Recall: {resultados[nome]['recall']:.4f}")
    print(f"F1-Score: {resultados[nome]['f1']:.4f}")
    print(f"ROC-AUC: {resultados[nome]['roc_auc']:.4f}")

print("\nTreinamento de todos os modelos concluído!")

## 6. Comparação de Modelos

In [ ]:
# Criando DataFrame com resultados
df_resultados = pd.DataFrame(resultados).T
print("Comparação de Modelos:")
print(df_resultados.sort_values('accuracy', ascending=False))

# Visualizando a comparação
df_resultados.plot(kind='bar', figsize=(14, 6))
plt.title('Comparação de Métricas entre Modelos')
plt.ylabel('Score')
plt.xlabel('Modelo')
plt.xticks(rotation=45, ha='right')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

## Sua Vez!

### Tarefas a Completar:

1. **Feature Engineering**: Crie novas features (ex: salario_por_idade, interacao_tempo_paginas)
2. **Otimização**: Use GridSearchCV para otimizar hiperparâmetros do melhor modelo
3. **Análise de Erros**: Analise os casos em que o modelo errou
4. **Interpretação**: Use feature importance para entender quais variáveis são mais importantes
5. **Curva ROC**: Plote a curva ROC para o melhor modelo

### Espaço para seu código:

In [ ]:
# Seu código aqui
